# Creating Trading Signals


The aim of this code file is to 

1. Create a dataframe to store trading signals
2. Calculate ratio of prices between asset 1 and asset 2 and store values 
3. Calculate z-score of ratio
4. Set upper and lower threshold as mean+- std
5. Signal:(flag column) short (-1) if zscore > threshold and long (+1) if zscore < threshold
    1. When we are long we buy one share of asset1 and sell asset2
    2. When we are short we sell one share of asset1 and buy asset2
    

# Importing Necessary Libraries

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm

from scipy.stats import zscore

random.seed(1)

In [2]:
# Creating a dataframe which will store all the trading signals 
df_trading_signals = pd.DataFrame()

# Chosen Pairs

1. Bank of America Corp and PNC
2. Morgan Stanley, U.S. Bankcorp
3. Commerzbank AG and Bank of America 

# Bank of America Corp and PNC

In [3]:
# Getting the data from Yahoo Finance
end = datetime.now().date() # takes today's date
start = (datetime.now() - relativedelta(years=1)).date() # takes date one year ago today

asset1 = yf.download("BAC", start=start, end=end, progress = False, group_by="ticker")["Adj Close"]
asset2 = yf.download("PNC", start=start, end=end, progress = False, group_by="ticker")["Adj Close"]


# Splitting into train and test
asset1_train, asset1_test, asset2_train, asset2_test = train_test_split(asset1, asset2, test_size = 0.2, random_state= 1, 
                                                                       shuffle = False)

# Creating a dataframe for the prices alone
df = pd.DataFrame()
df["BAC"] = asset1_test
df["PNC"] = asset2_test

# Calculating the ratio of prices and the zscores
df["price_ratio"] = df["BAC"]/df["PNC"]
df["price_ratio_zscores"] = zscore(df['price_ratio'])


# Settign Upper limit and Lower Limit as Mean +- Std
UL = df["price_ratio_zscores"].mean() + df["price_ratio_zscores"].std()
LL = df["price_ratio_zscores"].mean() - df["price_ratio_zscores"].std()

# Creating trading signals
# Long if zscore > UL 
# Short if zscore < LL
df["trading_signals"] = np.where(df["price_ratio_zscores"] > UL, -1, 0)
df["trading_signals"] = np.where(df["price_ratio_zscores"] < LL, 1, df["trading_signals"])


df.head()

,BAC,PNC,price_ratio,price_ratio_zscores,trading_signals
Date,,,,,
2022-08-11,35.674946,170.253983,0.209540,-0.256736,0
2022-08-12,36.062393,172.917603,0.208552,-0.445307,0
2022-08-15,36.012718,172.224457,0.209103,-0.340061,0
2022-08-16,36.400166,173.323578,0.210013,-0.166336,0
2022-08-17,36.171673,171.640244,0.210741,-0.027179,0


In [4]:
# Checking how many times we received signals
df["trading_signals"].value_counts()

 0    38
 1     9
-1     4
Name: trading_signals, dtype: int64

##### Saving the values in the overall dataframe

In [5]:
df_trading_signals["Price-PNC"] = df["PNC"]
df_trading_signals["Price-BAC"] = df["BAC"]
df_trading_signals["BAC-PNC"] = df["price_ratio_zscores"]
df_trading_signals["BAC-PNC-signals"] = df["trading_signals"]

# Morgan Stanley and US Bankcorp

In [6]:
# Getting the data from Yahoo Finance
end = datetime.now().date() # takes today's date
start = (datetime.now() - relativedelta(years=1)).date() # takes date one year ago today

asset1 = yf.download("MS-PK", start=start, end=end, progress = False, group_by="ticker")["Adj Close"]
asset2 = yf.download("USB-PH", start=start, end=end, progress = False, group_by="ticker")["Adj Close"]


# Splitting into train and test
asset1_train, asset1_test, asset2_train, asset2_test = train_test_split(asset1, asset2, test_size = 0.2, random_state= 1, 
                                                                       shuffle = False)


df = pd.DataFrame()
df["MS-PK"] = asset1_test
df["USB-PH"] = asset2_test


df["price_ratio"] = df["MS-PK"]/df["USB-PH"]
df["price_ratio_zscores"] = zscore(df['price_ratio'])


UL = df["price_ratio_zscores"].mean() + df["price_ratio_zscores"].std()
LL = df["price_ratio_zscores"].mean() - df["price_ratio_zscores"].std()


df["trading_signals"] = np.where(df["price_ratio_zscores"] > UL, -1, 0)
df["trading_signals"] = np.where(df["price_ratio_zscores"] < LL, 1, df["trading_signals"])


df.head()

,MS-PK,USB-PH,price_ratio,price_ratio_zscores,trading_signals
Date,,,,,
2022-08-11,25.390839,20.813704,1.219910,-1.566423,1
2022-08-12,25.380985,20.863144,1.216547,-1.708225,1
2022-08-15,25.440104,20.962021,1.213628,-1.831263,1
2022-08-16,25.400692,20.952133,1.212320,-1.886425,1
2022-08-17,25.144518,20.645611,1.217911,-1.650691,1


In [7]:
# Checking how many times we received signals
df["trading_signals"].value_counts()

 0    31
 1    11
-1     9
Name: trading_signals, dtype: int64

##### Saving the values in the overall dataframe

In [8]:
df_trading_signals["Price-MS-PK"] = df["MS-PK"]
df_trading_signals["Price-USB-PH"] = df["USB-PH"]
df_trading_signals["MS-PK-USB-PH"] = df["price_ratio_zscores"]
df_trading_signals["MS-PK-USB-PH-signals"] = df["trading_signals"]

# Commerzbank AG and Bank of America Corp

In [9]:
# Getting the data from Yahoo Finance
end = datetime.now().date() # takes today's date
start = (datetime.now() - relativedelta(years=1)).date() # takes date one year ago today

asset1 = yf.download("CRZBY", start=start, end=end, progress = False, group_by="ticker")["Adj Close"]
asset2 = yf.download("BAC", start=start, end=end, progress = False, group_by="ticker")["Adj Close"]


# Splitting into train and test
asset1_train, asset1_test, asset2_train, asset2_test = train_test_split(asset1, asset2, test_size = 0.2, random_state= 1, 
                                                                       shuffle = False)


df = pd.DataFrame()
df["CRZBY"] = asset1_test
df["BAC"] = asset2_test


df["price_ratio"] = df["CRZBY"]/df["BAC"]
df["price_ratio_zscores"] = zscore(df['price_ratio'])


UL = df["price_ratio_zscores"].mean() + df["price_ratio_zscores"].std()
LL = df["price_ratio_zscores"].mean() - df["price_ratio_zscores"].std()


df["trading_signals"] = np.where(df["price_ratio_zscores"] > UL, -1, 0)
df["trading_signals"] = np.where(df["price_ratio_zscores"] < LL, 1, df["trading_signals"])


df.head()

,CRZBY,BAC,price_ratio,price_ratio_zscores,trading_signals
Date,,,,,
2022-08-11,7.08,35.674946,0.198459,-0.937327,0
2022-08-12,7.34,36.062393,0.203536,-0.673181,0
2022-08-15,7.21,36.012718,0.200207,-0.846369,0
2022-08-16,7.27,36.400166,0.199724,-0.871479,0
2022-08-17,7.14,36.171673,0.197392,-0.992813,0


In [10]:
# Checking how many times we received signals
df["trading_signals"].value_counts()

 0    37
 1    10
-1     4
Name: trading_signals, dtype: int64

##### Saving the values in the overall dataframe

In [11]:
df_trading_signals["Price-CRZBY"] = df["CRZBY"]
df_trading_signals["Price-BAC"] = df["BAC"]
df_trading_signals["CRZBY-BAC"] = df["price_ratio_zscores"]
df_trading_signals["CRZBY-BAC-signals"] = df["trading_signals"]

# Trading Signals for Pairs

In [12]:
df_trading_signals

,Price-PNC,Price-BAC,BAC-PNC,BAC-PNC-signals,Price-MS-PK,Price-USB-PH,MS-PK-USB-PH,MS-PK-USB-PH-signals,Price-CRZBY,CRZBY-BAC,CRZBY-BAC-signals
Date,,,,,,,,,,,
2022-08-11,170.253983,35.674946,-0.256736,0,25.390839,20.813704,-1.566423,1,7.08,-0.937327,0
2022-08-12,172.917603,36.062393,-0.445307,0,25.380985,20.863144,-1.708225,1,7.34,-0.673181,0
2022-08-15,172.224457,36.012718,-0.340061,0,25.440104,20.962021,-1.831263,1,7.21,-0.846369,0
2022-08-16,173.323578,36.400166,-0.166336,0,25.400692,20.952133,-1.886425,1,7.27,-0.871479,0
2022-08-17,171.640244,36.171673,-0.027179,0,25.144518,20.645611,-1.650691,1,7.14,-0.992813,0
2022-08-18,171.798691,36.042522,-0.207921,0,25.326796,20.774153,-1.598480,1,7.01,-1.143655,1
2022-08-19,169.115250,35.247761,-0.469754,0,25.124811,20.596174,-1.567777,1,6.69,-1.387807,1
2022-08-22,164.827728,34.492737,-0.309116,0,24.977018,20.269878,-1.047237,1,6.29,-1.774963,1
2022-08-23,164.857422,34.184765,-0.673193,0,24.691284,19.577736,0.174175,0,6.23,-1.780805,1


# Saving data

In [13]:
# Saving the data to a .csv file for further use
df_trading_signals.to_csv("data/trading_signals.csv")